In [ ]:
pip install pandas matplotlib


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 16.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 60.1 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 36.1 MB/s eta 0:00:01
     |████████████████████████████████| 253 kB 34.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 41.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 22.4 MB/s eta 0:00:011


In [ ]:

import pandas as pd

file_path = '../setagayasfcjitaku.csv'
df = pd.read_csv(file_path)

# Display the first few rows to understand its structure
df.head()


zsh:1: command not found: pip


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Plot with value labels on each bar
import numpy as np

# Prepare data for bar chart
summary_df = df_clean[['AS', 'SFC_SUM', 'jitaku_SUM', 'setagaya_SUM']].dropna()
summary_df.set_index('AS', inplace=True)

# Transpose for plotting
summary_df = summary_df.transpose()

# Plot
plt.figure(figsize=(14, 7))
x = np.arange(len(summary_df.columns))
bar_width = 0.25

for i, location in enumerate(summary_df.index):
    plt.bar(x + i * bar_width, summary_df.loc[location], width=bar_width, label=location)
    # Add value labels
    for j, val in enumerate(summary_df.loc[location]):
        plt.text(x[j] + i * bar_width, val + 0.3, str(int(val)), ha='center', va='bottom', fontsize=8)

plt.xticks(x + bar_width, summary_df.columns, rotation=45)
plt.title('AS Rule Distribution per Location')
plt.xlabel('AS (Wi-Fi Rule)')
plt.ylabel('Rule Count')
plt.legend()
plt.tight_layout()
plt.grid(True, axis='y')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Clean up and extract relevant data
df_clean = df.iloc[1:].copy()
df_clean.columns = ['AS', 'SFC_1', 'SFC_2', 'SFC_3', 'SFC_4', 'SFC_5', 'SFC_SUM',
                    'jitaku_1', 'jitaku_2', 'jitaku_3', 'jitaku_4', 'jitaku_5', 'jitaku_SUM',
                    'setagaya_1', 'setagaya_2', 'setagaya_3', 'setagaya_4', 'setagaya_5', 'setagaya_SUM']

# Convert numeric columns to float
numeric_cols = df_clean.columns[1:]
df_clean[numeric_cols] = df_clean[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Create a melted DataFrame for distribution plot
df_melted = df_clean.melt(id_vars='AS', value_vars=['SFC_SUM', 'jitaku_SUM', 'setagaya_SUM'],
                          var_name='Location', value_name='Count')

# Drop NaNs
df_melted.dropna(inplace=True)

# Plot
plt.figure(figsize=(12, 6))
for location in df_melted['Location'].unique():
    subset = df_melted[df_melted['Location'] == location]
    plt.hist(subset['Count'], bins=10, alpha=0.5, label=location)

plt.title('AS Rule Distribution per Location (SFC, Jitaku, Setagaya)')
plt.xlabel('Rule Count')
plt.ylabel('Number of AS Entries')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Sort AS by total rule count (sum across SFC, jitaku, setagaya)
summary_df_sorted = summary_df.loc[:, summary_df.sum(axis=0).sort_values(ascending=False).index]

# Plot again with sorted AS order
plt.figure(figsize=(14, 7))
x = np.arange(len(summary_df_sorted.columns))
bar_width = 0.25

for i, location in enumerate(summary_df_sorted.index):
    plt.bar(x + i * bar_width, summary_df_sorted.loc[location], width=bar_width, label=location)
    for j, val in enumerate(summary_df_sorted.loc[location]):
        plt.text(x[j] + i * bar_width, val + 0.3, str(int(val)), ha='center', va='bottom', fontsize=8)

plt.xticks(x + bar_width, summary_df_sorted.columns, rotation=45)
plt.title('AS Rule Distribution per Location (Sorted by Total Count)')
plt.xlabel('AS (Wi-Fi Rule)')
plt.ylabel('Rule Count')
plt.legend()
plt.tight_layout()
plt.grid(True, axis='y')
plt.show()


NameError: name 'summary_df' is not defined

In [ ]:
import plotly.express as px

# Prepare data for Sunburst chart
sunburst_data = []

for idx, row in df_clean.iterrows():
    as_name = row['AS']
    for location in ['SFC_SUM', 'jitaku_SUM', 'setagaya_SUM']:
        count = row[location]
        if pd.notna(count) and count > 0:
            sunburst_data.append({
                'AS': as_name,
                'Location': location.replace('_SUM', ''),
                'Count': count
            })

sunburst_df = pd.DataFrame(sunburst_data)

# Create sunburst chart
fig = px.sunburst(
    sunburst_df,
    path=['Location', 'AS'],
    values='Count',
    title='AS Rule Distribution by Location (Sunburst View)',
    color='Location'
)

fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()


In [ ]:
# Prepare data again for static sunburst-like chart
sunburst_data = df_clean[['AS', 'SFC_SUM', 'jitaku_SUM', 'setagaya_SUM']].copy()
sunburst_data = sunburst_data.melt(id_vars='AS', var_name='Location', value_name='Count')
sunburst_data.dropna(inplace=True)

# Aggregate total counts per (Location, AS)
grouped_data = sunburst_data.groupby(['Location', 'AS']).sum().reset_index()

# Create a pie chart for each Location
locations = grouped_data['Location'].unique()

# Plot
fig, axs = plt.subplots(1, len(locations), figsize=(18, 6))

for i, loc in enumerate(locations):
    subset = grouped_data[grouped_data['Location'] == loc]
    axs[i].pie(subset['Count'], labels=subset['AS'], autopct='%1.1f%%', startangle=90)
    axs[i].axis('equal')
    axs[i].set_title(f'{loc} AS Distribution')

plt.suptitle('AS Rule Distribution per Location (Pie Charts)')
plt.tight_layout()
plt.show()
